In [17]:
import matplotlib.pyplot as plt

In [2]:
pixels = 128

In [3]:
timetagger_pixel_counter._tagger.reset()
timetagger_pixel_counter.configure(pixels=pixels)

0

In [19]:
count_rates = timetagger_pixel_counter.get_cleaned_count_rate()
forward, backward = timetagger_pixel_counter.get_forward_backward_counts(count_rates)

fig, (ax, ax1) = plt.subplots(ncols=2, sharey="row", figsize=(10, 10))

img_forward = ax.imshow(forward, origin="lower")
ax.set_title("Forward")
fig.colorbar(img_forward, ax=ax, shrink=0.4)

img_backward = ax1.imshow(backward, origin="lower")
ax1.set_title("Backward")
fig.colorbar(img_backward, ax=ax1, shrink=0.4)


plt.show()
# plt.close()
fig.show()

fig.savefig("timetagger_pixel_scanner.jpg", dpi=300)